## 🪄 Install `raga-testing-platform` library

In [ ]:
pip install raga-testing-platform==1.1.51

### Put your `ACCESS KEY & SECRET KEY`

In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "Paste_your_access_key_here"
SECRET_KEY = "Paste_your_secret_key_here"
HOST = "https://backend.prod3.ragaai.ai"

### Import All raga lib from raga module

In [ ]:
from raga import *
import datetime

### 🪄 Set up a Run

In [ ]:
project_name = "Object Detection"
run_name = f"FasterRCNN-Tests-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

### 👟 Run Data Drift Detection

In [ ]:
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

rules = DriftDetectionRules()
rules.add(type="drift_detection", dist_metric="Mahalanobis", _class="ALL", threshold=2)

edge_case_detection = data_drift_detection(test_session=test_session,
                                            test_name=f"Drift-Detection-Test-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}",
                                            train_dataset_name="bdd_train_1",
                                            field_dataset_name="bdd_val_1",
                                            train_embed_col_name="ImageVectorsM1",
                                            field_embed_col_name = "ImageVectorsM1",
                                            level = "image",
                                            rules = rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Outlier Detection

In [ ]:
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=25)

edge_case_detection = data_drift_detection(test_session=test_session,
                                            test_name=f"Outlier_Detection_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}",
                                            dataset_name="bdd_val_2",
                                            embed_col_name = "ImageVectorsM1",
                                            output_type = "outlier_detection",
                                            rules = rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Data Leakage Test

In [ ]:
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

rules = DLRules()
rules.add(metric = 'overlapping_samples', metric_threshold = 0.92)

edge_case_detection = data_leakage_test(test_session=test_session,
                                            test_name=f"Data-Leakage-Test-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}",
                                            train_dataset_name="bdd_train_2",
                                            dataset_name="bdd_test_1",
                                            type = "data_leakage",
                                            output_type="image_data",
                                            train_embed_col_name="ImageVectorsM1",
                                            embed_col_name = "ImageVectorsM1",
                                            rules = rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Failure Mode Analysis - Cluster Level

In [ ]:
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.4, metric_threshold=0.60, iou_threshold=0.4, label="ALL")
rules.add(metric="F1Score", conf_threshold=0.4, metric_threshold=0.60, iou_threshold=0.4, label="ALL")
rules.add(metric="Recall", conf_threshold=0.4, metric_threshold=0.60, iou_threshold=0.4, label="ALL")

cls_default = clustering(test_session=test_session, dataset_name = "bdd_val_3", method="k-means", embedding_col="ImageVectorsM1", level="image", args= {"numOfClusters": 6}, force=True)

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = "bdd_val_3",
                                            test_name = f"Failure_Mode_Analysis_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}",
                                            model = "FasterRCNN",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="object_detection",
                                            type="embedding",
                                            clustering=cls_default
                                            )

test_session.add(edge_case_detection)
test_session.run()

Please switch to https://prod3.ragaai.ai/home/project/runs?projectId=51